In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.9 MB/s eta 0:00:00


In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 4.7 MB/s eta 0:00:00
  Created wheel for alembic: filename=alembic-1.9.3-py3-none-any.whl size=210578 sha256=619ad3a47438a28fa517b737e12aca7c7b1912c269c85de5ec88fe93b5ce43cd
  Stored in directory: /root/.cache/pip/wheels/4f/5b/74/1ce76c741045a20a9394fa0c8f8fcc45b6f2112ab8ec1a78ff
Successfully built alembic


In [4]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_absolute_error, roc_auc_score, f1_score, confusion_matrix
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from catboost import CatBoostClassifier, CatBoostRegressor

In [5]:
dpath = 'drive/MyDrive/Seculayer/OSC/data/'

train = pd.read_csv(dpath + 'train.csv')
test = pd.read_csv(dpath + 'test.csv')

In [6]:
guide_list = ['AL',"CA","BA","Y_LABEL"]

for item in test.columns:
    guide_list.append(item)

In [7]:
teacher_X = train[guide_list]


In [8]:
teacher_X = teacher_X.drop(['ID'],axis=1)

# teacher_y = real['Y_LABEL']

In [9]:
le1 = LabelEncoder()
le2 = LabelEncoder()

teacher_X['COMPONENT_ARBITRARY_cat'] = le1.fit_transform(teacher_X['COMPONENT_ARBITRARY'])
teacher_X['YEAR_cat'] = le2.fit_transform(teacher_X['YEAR'])
teacher_X.drop(['YEAR','COMPONENT_ARBITRARY'],axis=1,inplace=True)



In [10]:
categorical_features = ['COMPONENT_ARBITRARY_cat','YEAR_cat']

In [11]:
teacher_X.dropna(axis=1,inplace=True)

In [12]:
teacher_y = teacher_X['Y_LABEL']
train_3 = teacher_X.copy()
teacher_X.drop('Y_LABEL',axis=1,inplace=True)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(teacher_X,teacher_y,test_size=0.3,random_state=42,stratify = teacher_y)

In [14]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 16)
  }
    
    model = CatBoostClassifier(**params_cat,eval_metric="F1")
    model.fit(X_train, y_train, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, cat_features = categorical_features, verbose = False)

    cat_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, cat_pred)
    
    return AUC

In [15]:
# Optuna 초모수 작업 시작

sampler = TPESampler(seed = 2023)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2023-02-09 00:09:32,095] A new study created in memory with name: cat_parameter_opt
<ipython-input-14-e8f0cc6b2dca>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
[I 2023-02-09 00:09:39,708] Trial 0 finished with value: 0.7443767599480932 and parameters: {'learning_rate': 0.009246234676674875, 'n_estimators': 902, 'max_depth': 11}. Best is trial 0 with value: 0.7443767599480932.
<ipython-input-14-e8f0cc6b2dca>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
[I 2023-02-09 00:09:46,422] Trial 1 finish

In [16]:
# 가장 좋은 초모수와 성능 확인

print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.772908687519157
Best trial : {'learning_rate': 0.8667288542150742, 'n_estimators': 676, 'max_depth': 3}


In [ ]:
{'learning_rate': 0.11833658084738576, 'n_estimators': 648, 'max_depth': 14}

In [17]:
# 위의 초모수 적용하고, StratifiedKFold을 이용해 모델 적합 
import pickle

n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((teacher_X.shape[0], 2))
cat_train = np.zeros((X_train.shape[0], 2))

for i, (i_trn, i_val) in enumerate(cv.split(teacher_X, teacher_y), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostClassifier(
        random_state = 39,
        learning_rate = 0.8667288542150742, 
        n_estimators = 676, 
        max_depth = 3,
        eval_metric="F1")

    optuna_cat.fit(teacher_X.loc[i_trn, :], teacher_y[i_trn], verbose = False, cat_features = categorical_features)

    cat_val[i_val, :] = optuna_cat.predict_proba(teacher_X.loc[i_val, :])
    cat_train += optuna_cat.predict_proba(X_train) / n_fold

    # 학습이 완료된 Teacher 모델 5개를 저장 
    with open('Teacher_model' + str(i) + '.pickle', 'wb') as fw:
        pickle.dump(optuna_cat, fw)

training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [18]:
# Teacher model에서 구한 예측불량률을 train 데이터에 넣기

train_3['model1_prob'] = cat_val[:, 1]
print(train_3.shape)

(14095, 23)


In [19]:
test.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN'],
      dtype='object')

In [20]:
guide_list = ['COMPONENT_ARBITRARY_cat','YEAR_cat']

In [21]:
student_list = []
for item in train_3.columns:
    if item in test.columns:
        student_list.append(item)
    if item in guide_list:
        student_list.append(item)

student_X  = train_3[student_list]
student_y = train_3['model1_prob']

In [22]:
X_train, X_val, y_train, y_val = train_test_split(student_X, student_y, test_size = 0.3, random_state = 39)


In [23]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 16)
  }
    
    model = CatBoostRegressor(**params_cat)
    model.fit(X_train, y_train, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, cat_features = categorical_features, verbose = False)

    cat_pred = model.predict(X_val)
    MAE = mean_absolute_error(y_val, cat_pred)
    
    return MAE

In [24]:
# Optuna 초모수 작업 시작

sampler = TPESampler(seed = 2023)
study = optuna.create_study(
    study_name = "cat_parameter_opt_reg",
    direction = "minimize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2023-02-09 01:14:25,151] A new study created in memory with name: cat_parameter_opt_reg
<ipython-input-23-b95a039dfa64>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
[I 2023-02-09 01:15:38,964] Trial 0 finished with value: 0.10570552225748231 and parameters: {'learning_rate': 0.009246234676674875, 'n_estimators': 902, 'max_depth': 11}. Best is trial 0 with value: 0.10570552225748231.
<ipython-input-23-b95a039dfa64>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
[I 2023-02-09 01:15:47,125] Trial 1 

KeyboardInterrupt: ignored

In [ ]:
'learning_rate': 0.021058832594574306, 'n_estimators': 509, 'max_depth': 9

In [25]:
test2 = test.drop(['ID'],axis=1)
test2['COMPONENT_ARBITRARY_cat'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_cat'] = le2.transform(test2['YEAR'])
test3 = test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
X_test = test3.copy()

In [26]:
import pickle

n_fold = 5
cv = KFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((student_X.shape[0]))
cat_test = np.zeros((X_test.shape[0]))

for i, (i_trn, i_val) in enumerate(cv.split(student_X, student_y), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostRegressor(
        random_state = 39,
        learning_rate = 0.021058832594574306, 
        n_estimators = 509, 
        max_depth = 9)

    optuna_cat.fit(student_X.loc[i_trn, :], student_y[i_trn], verbose = False, cat_features = categorical_features)

    cat_val[i_val] = optuna_cat.predict(student_X.loc[i_val, :])
    cat_test += optuna_cat.predict(X_test) / n_fold

    # 학습이 완료된 Teacher 모델 5개를 저장 
    with open('Teacher_model' + str(i) + '.pickle', 'wb') as fw:
        pickle.dump(optuna_cat, fw)

training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [27]:
scores = []
TP = []
FP = []
FN = []
TN = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = cat_val
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(teacher_y, pred)
    scores.append(score)
    TP.append(confusion_matrix(teacher_y, pred)[0][0])
    FN.append(confusion_matrix(teacher_y, pred)[0][1])
    FP.append(confusion_matrix(teacher_y, pred)[1][0])
    TN.append(confusion_matrix(teacher_y, pred)[1][1])
    

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
temp3 = pd.DataFrame(TP, columns = ['TP'])
temp4 = pd.DataFrame(FP, columns = ['FP'])
temp5 = pd.DataFrame(FN, columns = ['FN'])
temp6 = pd.DataFrame(TN, columns = ['TN'])
scores = pd.concat([temp1, temp2, temp3, temp4, temp5, temp6], axis = 1)
scores

,threshold,score,TP,FP,FN,TN
0,0.00,0.157978,68,0,12824,1203
1,0.02,0.167387,1165,22,11727,1181
2,0.04,0.188686,3929,144,8963,1059
3,0.06,0.218696,6911,321,5981,882
4,0.08,0.243453,9232,529,3660,674
5,0.10,0.262055,10779,703,2113,500
6,0.12,0.262789,11616,828,1276,375
7,0.14,0.237762,12079,931,813,272
8,0.16,0.196842,12382,1016,510,187
9,0.18,0.166367,12563,1064,329,139


In [28]:
scores = []
for threshold in range(100) :
    threshold = threshold / 100
    pred = cat_val
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(teacher_y, pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.99, 100), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores.loc[: 50, :]

,threshold,score
0,0.00,0.157978
1,0.01,0.160316
2,0.02,0.167387
3,0.03,0.178095
4,0.04,0.188686
5,0.05,0.204001
6,0.06,0.218696
7,0.07,0.235188
8,0.08,0.243453
9,0.09,0.257953


In [29]:
scores.loc[scores['score'] == scores['score'].max(), :]

,threshold,score
11,0.11,0.26851


In [30]:
from collections import Counter
answer = np.zeros(cat_test.shape[0])

for i in range(cat_test.shape[0]) :
  if cat_test[i] >= 0.11 :
    answer[i] = 1
    
answer = answer.astype('int64')
print(Counter(answer))

Counter({0: 5174, 1: 867})


In [31]:
submission_preds = answer
submission = pd.read_csv(dpath+'sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv(dpath + 'submission/KD_with_core_opt_version2_submission.csv', index = False)

In [36]:
len(cat_test)

6041

In [38]:
submission['Y_LABEL'] = cat_test 
submission.to_csv(dpath+'for_ensemble/KD_with_core_opt_version2_submission.csv', index = False)